In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import sys
import time
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler

import numpy as np
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from matplotlib import pyplot as plt
from PIL import Image
from pathlib import Path
import yaml

print("=== Faster R-CNN Training Setup ===")
print(f'Python version: {sys.version.split()[0]}')
print(f'PyTorch version: {torch.__version__}')
print(f'Torchvision version: {torchvision.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'CUDA device: {torch.cuda.get_device_name(0)}')
    print(f'CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

In [ ]:
# Helper function to load data configuration from yaml_path
# Returns a dict with paths, number of classes, and class names
def load_data_config(yaml_path):
    yaml_path = Path(yaml_path)
    
    if not yaml_path.exists():
        raise FileNotFoundError(f"data.yaml not found at {yaml_path}")
    
    with open(yaml_path, 'r') as file:
        config = yaml.safe_load(file)
    
    print(f"Loaded data configuration from {yaml_path}")
    print(f"Number of classes: {config['nc']}")
    print(f"Class names: {config['names']}")
    
    return config

In [ ]:
# Configuration class to store all hyperparameters and settings
class Config:
    # Path Configuration
    DATA_ROOT = Path("./data")
    DATA_YAML = "./data/data.yaml"
    
    # Print current working directory for debugging
    print(f"Current working directory: {Path.cwd()}")
    print(f"Looking for data.yaml at: {Path(DATA_YAML).absolute()}")
    print(f"Looking for data root at: {DATA_ROOT.absolute()}")
    
    # Training mode selection
    # - 'quick_test': Fast iteration for testing pipeline
    # - 'full': Complete training on all data
    TRAINING_MODE = 'full'
    
    # Subset configuration based on training mode
    if TRAINING_MODE == 'quick_test':
        USE_SUBSET = 500
        SUBSET_VALID = 100
        NUM_EPOCHS = 5
        print(f"\n⚡ QUICK TEST MODE: Using {USE_SUBSET} train images, {SUBSET_VALID} valid images, {NUM_EPOCHS} epochs")
    else:
        USE_SUBSET = None
        SUBSET_VALID = None
        NUM_EPOCHS = 12
        print(f"\n🚀 FULL TRAINING MODE: Using all images, {NUM_EPOCHS} epochs")
    
    # Load dataset configuration from YAML file
    try:
        data_config = load_data_config(DATA_YAML)
        NUM_CLASSES = data_config['nc'] + 1  # plus 1 for background class
        CLASS_NAMES = ['background'] + data_config['names']  # Add background as class 0
        
        # Verify paths exist
        if not DATA_ROOT.exists():
            print(f"Warning: Data root directory not found at {DATA_ROOT.absolute()}")
        else:
            print(f"✓ Data root verified at: {DATA_ROOT.absolute()}")
            
            # Check train/valid/test directories
            for split in ['train', 'valid', 'test']:
                img_path = DATA_ROOT / split / 'images'
                lbl_path = DATA_ROOT / split / 'labels'
                if img_path.exists() and lbl_path.exists():
                    num_images = len(list(img_path.glob('*.jpg')))
                    print(f"  ✓ {split}: {num_images} images found")
                else:
                    print(f"  ✗ Warning: {split} directory incomplete or missing")
                    print(f"    Expected images at: {img_path.absolute()}")
                    print(f"    Expected labels at: {lbl_path.absolute()}")
                    
    except FileNotFoundError as e:
        print(f"\n❌ Error loading data configuration: {e}")
        print(f"Current working directory: {Path.cwd()}")
        print(f"Tried to load from: {Path(DATA_YAML).absolute()}")
        print("\nPlease ensure:")
        print("  1. You're running the notebook from the dl_pipeline/ directory, OR")
        print("  2. Adjust DATA_YAML path to point to your data.yaml location")
        NUM_CLASSES = None
        CLASS_NAMES = None
    
    # Model configuration
    BACKBONE = 'resnet50'
    PRETRAINED = True
    
    # Training hyperparameters
    BATCH_SIZE = 4
    LEARNING_RATE = 0.002
    WEIGHT_DECAY = 0.0005
    MOMENTUM = 0.9  
    
    # Image processing
    IMG_SIZE = (416, 416)
    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]
    
    # Training settings
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    NUM_WORKERS = 6
    PRINT_FREQ = 5                      # Print training stats every 5 batches
    SAVE_FREQ = 2                       # Save checkpoint every 2 epochs
    
    # Advanced training features
    GRADIENT_ACCUMULATION_STEPS = 2     # Simulate batch_size of 8 (4*2)
    MIXED_PRECISION = True              # Use FP16 for faster training & less memory
    
    # Learning rate scheduler settings
    LR_STEP_SIZE = 4                    # Reduce LR every 4 epochs
    LR_GAMMA = 0.3                      # Multiply LR by 0.3 when stepping

# Create config instance
config = Config()
print(f"\n=== Configuration Summary ===")
print(f"Hardware: RTX 3070 Ti (8GB VRAM) + i7-12700")
print(f"Training on device: {config.DEVICE}")
print(f"Number of classes: {config.NUM_CLASSES}")
if config.CLASS_NAMES:
    print(f"Class names: {config.CLASS_NAMES}")
print(f"\nTraining Settings:")
print(f"  Batch size: {config.BATCH_SIZE} (effective: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS} with gradient accumulation)")
print(f"  Image size: {config.IMG_SIZE}")
print(f"  Epochs: {config.NUM_EPOCHS}")
print(f"  Workers: {config.NUM_WORKERS}")
print(f"  Mixed precision: {config.MIXED_PRECISION}")
print(f"\nEstimated training time:")
if config.TRAINING_MODE == 'quick_test':
    print(f"  ⚡ Quick test: ~30-45 minutes")
else:
    print(f"  🚀 Full training: ~2-3 hours")

In [ ]:
# Custom Dataset class for AgroPest-12 dataset
    # This class handles:
    #   - Loading images from the dataset
    #   - Converting YOLO format annotations to Faster R-CNN format
    #   - Applying data augmentations
    #   - Returning properly formatted tensors for training
class AgroPestDataset(Dataset):
    # Initializes the dataset
    # Args: 
    #   - data_dir (str/Path): Root directory containing train/valid/test folders
    #   - split (str): Which split to use ('train', 'valid', or 'test')
    #   - transform (callable): Optional transform to apply to images
    def __init__(self, data_dir, split='train', transform=None):
        self.data_dir = Path(data_dir)
        self.split = split
        self.transform = transform
        
        # Set paths for images and labels
        self.image_dir = self.data_dir / split / 'images'
        self.label_dir = self.data_dir / split / 'labels'
        
        # Verify that image and label directories exist
        if not self.image_dir.exists():
            raise FileNotFoundError(f"Image directory not found: {self.image_dir}")
        if not self.label_dir.exists():
            raise FileNotFoundError(f"Label directory not found: {self.label_dir}")
        
        # Get list of all image files
        self.image_paths = list(self.image_dir.glob('*.jpg'))
        
        # Use subset for faster training/testing if specified
        if hasattr(config, 'USE_SUBSET') and config.USE_SUBSET is not None:
            if split == 'train' and config.USE_SUBSET:
                original_count = len(self.image_paths)
                self.image_paths = self.image_paths[:config.USE_SUBSET]
                print(f"Using train subset: {len(self.image_paths)}/{original_count} images")
            elif split == 'valid' and hasattr(config, 'SUBSET_VALID') and config.SUBSET_VALID:
                original_count = len(self.image_paths)
                self.image_paths = self.image_paths[:config.SUBSET_VALID]
                print(f"Using valid subset: {len(self.image_paths)}/{original_count} images")
        
        print(f"Loaded {len(self.image_paths)} images from {split} split")
    
    # Return the total number of samples in the dataset
    def __len__(self):
        return len(self.image_paths)
    
    # Get a single sample from the dataset
    # Args: 
    #   - idx (int): Index of the sample to retrieve
    # Returns: 
    #   - tuple: (image, target) where image is a tensor and target is a dict
    def __getitem__(self, idx):        
        img_path = self.image_paths[idx]
        
        # Load image
        image = Image.open(img_path).convert('RGB')
        
        # Load annotations
        label_path = self.label_dir / (img_path.stem + '.txt')
        
        boxes, labels = self._load_yolo_annotations(label_path, image.size)
        
        # Prepare target dictionary (format required by Faster R-CNN)
        target = {
            'boxes': torch.as_tensor(boxes, dtype=torch.float32),
            'labels': torch.as_tensor(labels, dtype=torch.int64),
            'image_id': torch.tensor([idx]),
            'area': self._calculate_area(boxes),
            'iscrowd': torch.zeros((len(boxes),), dtype=torch.int64)
        }
        
        # Apply transforms if specified
        if self.transform:
            image = self.transform(image)
        
        return image, target
    
    # Load and convert YOLO format annotations to Faster R-CNN format
    # Args:
    #   - label_path (Path): Path to the YOLO label file
    #   - img_size (tuple): (width, height) of the image
    # Returns:
    #   - tuple: (boxes, labels) as lists
    def _load_yolo_annotations(self, label_path, img_size):
        # YOLO format: class_id center_x center_y width height (normalized 0-1)
        # Faster R-CNN format: [x_min, y_min, x_max, y_max] (absolute pixels)
        
        boxes = []
        labels = []
        
        img_width, img_height = img_size
        
        # Check if label file exists
        if not label_path.exists():
            # Return empty annotations for images without labels
            return [], []
        
        # Read YOLO format annotations
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            line = line.strip()
            if not line:  # Skip empty lines
                continue
                
            # Parse YOLO format: class_id center_x center_y width height
            parts = line.split()
            if len(parts) != 5:
                continue  # Skip malformed lines
                
            class_id = int(parts[0])
            center_x = float(parts[1])
            center_y = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            
            # Convert normalized coordinates to absolute coordinates
            center_x_abs = center_x * img_width
            center_y_abs = center_y * img_height
            width_abs = width * img_width
            height_abs = height * img_height
            
            # Convert center format to corner format
            x_min = center_x_abs - width_abs / 2
            y_min = center_y_abs - height_abs / 2
            x_max = center_x_abs + width_abs / 2
            y_max = center_y_abs + height_abs / 2
            
            # Ensure coordinates are within image bounds
            x_min = max(0, x_min)
            y_min = max(0, y_min)
            x_max = min(img_width, x_max)
            y_max = min(img_height, y_max)
            
            # Only add valid boxes (with positive area)
            if x_max > x_min and y_max > y_min:
                boxes.append([x_min, y_min, x_max, y_max])
                labels.append(class_id + 1)  # Add 1 because Faster R-CNN uses 1-indexed labels (0 is background)
        
        return boxes, labels
    
    # Calculate area of bounding boxes
    def _calculate_area(self, boxes):
        if len(boxes) == 0:
            return torch.zeros((0,), dtype=torch.float32)
        
        boxes_tensor = torch.as_tensor(boxes, dtype=torch.float32)
        return (boxes_tensor[:, 2] - boxes_tensor[:, 0]) * (boxes_tensor[:, 3] - boxes_tensor[:, 1])

In [ ]:
# Get data transforms for training or validation
# Args:
#  - split (str): 'train' for training transforms, anything else for validation
# Returns:
#  - torchvision.transforms.Compose: Composed transforms
def get_transforms(split='train'):
    if split == 'train':
        # Training transforms with data augmentation
        return transforms.Compose([
            transforms.Resize(config.IMG_SIZE),  # Resize to fixed size
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color augmentation
            transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip
            transforms.ToTensor(),  # Convert PIL image to tensor
            transforms.Normalize(mean=config.MEAN, std=config.STD)  # Normalize with ImageNet stats
        ])
    else:
        # Validation transforms without augmentation
        return transforms.Compose([
            transforms.Resize(config.IMG_SIZE),  # Resize to fixed size
            transforms.ToTensor(),  # Convert PIL image to tensor
            transforms.Normalize(mean=config.MEAN, std=config.STD)  # Normalize with ImageNet stats
        ])

In [ ]:
# Create a Faster R-CNN model with custom number of classes
# Args:
#   - num_classes (int): Number of classes including background
# Returns:
#   - torch.nn.Module: Faster R-CNN model
def create_faster_rcnn_model(num_classes):
    # How Faster R-CNN works:
    # 1. Backbone (ResNet-50): Extracts features from input image
    # 2. FPN (Feature Pyramid Network): Combines features at different scales
    # 3. RPN (Region Proposal Network): Generates potential object locations
    # 4. ROI Head: Classifies proposals and refines bounding boxes
    
    # Load pre-trained Faster R-CNN model with ResNet-50 backbone and FPN
    # This model is pre-trained on COCO dataset (80 classes + background)
    from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

    if config.PRETRAINED:
        weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    else:
        weights = None

    model = fasterrcnn_resnet50_fpn(weights=weights)
    
    # Get the number of input features for the classifier
    # The classifier head takes features from the ROI pooling layer
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one for our number of classes
    # FastRCNNPredictor includes both classification and bounding box regression
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    print(f"Created Faster R-CNN model with {num_classes} classes")
    print(f"Backbone: ResNet-50 + FPN")
    print(f"Classifier input features: {in_features}")
    
    return model

In [ ]:
# Custom collate function to handle variable number of objects per image
# Args:
#  - batch (list): List of (image, target) tuples
# Returns:
#   - tuple: (images, targets) as lists
def collate_fn(batch):
    # Faster R-CNN needs a special collate function because:
    # - Different images have different numbers of objects
    # - We can't stack tensors with different shapes
    # - We need to return lists of images and targets

    images = []
    targets = []
    
    for image, target in batch:
        images.append(image)
        targets.append(target)
    
    return images, targets

In [ ]:
# Train the model for one epoch
# Args:
#   - model: The Faster R-CNN model
#   - optimizer: Optimizer for updating model weights
#   - data_loader: DataLoader for training data
#   - device: Device to run training on (CPU/GPU)
#   - epoch: Current epoch number (for logging)
#   - print_freq: Frequency of printing training stats
# Returns:
#   - float: Average loss for the epoch
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=5):
    model.train()
    
    # Initialize mixed precision scaler
    if config.MIXED_PRECISION:
        try:
            scaler = GradScaler("cuda")
        except TypeError:
            scaler = GradScaler()
    else:
        scaler = None
    
    losses = []
    optimizer.zero_grad()
    
    for batch_idx, (images, targets) in enumerate(data_loader):
        # Move images and targets to device (GPU)
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Forward pass with mixed precision if enabled
        if config.MIXED_PRECISION and scaler is not None:
            try:
                with autocast("cuda"):
                    loss_dict = model(images, targets)
            except AssertionError as e:
                if "Expected target boxes to be a tensor" in str(e):
                    print(f"Skipping batch with empty annotations")
                    continue  # Skip this batch
                else:
                    raise e  # Re-raise if it's a different error
        else:
            loss_dict = model(images, targets)
        
        # Calculate total loss
        total_loss = sum(loss for loss in loss_dict.values())
        
        # Scale loss for gradient accumulation
        total_loss = total_loss / config.GRADIENT_ACCUMULATION_STEPS
        
        # Backward pass
        if config.MIXED_PRECISION and scaler is not None:
            scaler.scale(total_loss).backward()  # Scaled backward pass
        else:
            total_loss.backward()
        
        losses.append(total_loss.item() * config.GRADIENT_ACCUMULATION_STEPS)  # Store unscaled loss
        
        # Update weights every GRADIENT_ACCUMULATION_STEPS
        if (batch_idx + 1) % config.GRADIENT_ACCUMULATION_STEPS == 0:
            if config.MIXED_PRECISION and scaler is not None:
                scaler.step(optimizer)  # Update weights with scaling
                scaler.update()  # Update scaler for next iteration
            else:
                optimizer.step()
            
            optimizer.zero_grad()  # Clear gradients
            
        if batch_idx % print_freq == 0 or batch_idx == 0:
            current_lr = optimizer.param_groups[0]['lr']
            memory_used = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0
            print(f'Epoch [{epoch}], Batch [{batch_idx}/{len(data_loader)}], '
                  f'Loss: {total_loss.item() * config.GRADIENT_ACCUMULATION_STEPS:.4f}, '
                  f'LR: {current_lr:.6f}, GPU Memory: {memory_used:.2f}GB')
            
            # Print individual loss components for debugging (less frequently)
            if batch_idx % (print_freq * 2) == 0:
                for loss_name, loss_value in loss_dict.items():
                    print(f'  {loss_name}: {loss_value.item():.4f}')
        
        # Clear cache periodically to prevent memory fragmentation
        if batch_idx % 10 == 0 and torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    return np.mean(losses)

# Validate the model (calculate validation loss)
# Args:
#   - model: Trained model
#   - data_loader: DataLoader with validation data
#   - device: Device to run validation on
# Returns:
#   - float: Average validation loss
def validate_model(model, data_loader, device):
    model.train()  # Keep in training mode to get loss values
    
    losses = []
    
    with torch.no_grad():  # Disable gradient calculation for efficiency
        for images, targets in data_loader:
            # Move to device
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            # Forward pass
            loss_dict = model(images, targets)
            total_loss = sum(loss for loss in loss_dict.values())
            
            losses.append(total_loss.item())
    
    return np.mean(losses)

In [ ]:

# Main training function to train the Faster R-CNN model
def train_faster_rcnn():
    # Create datasets
    print("Loading datasets...")
    train_dataset = AgroPestDataset(
        config.DATA_ROOT, 
        split='train',
        transform=get_transforms('train')
    )
    
    valid_dataset = AgroPestDataset(
        config.DATA_ROOT,
        split='valid', 
        transform=get_transforms('valid')
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=True,           # Shuffle training data
        num_workers=config.NUM_WORKERS,
        collate_fn=collate_fn   # Use custom collate function
    )
    
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=False,          # Don't shuffle validation data
        num_workers=config.NUM_WORKERS,
        collate_fn=collate_fn
    )
    
    print(f"Train batches: {len(train_loader)}")
    print(f"Valid batches: {len(valid_loader)}")
    
    # Create model
    model = create_faster_rcnn_model(config.NUM_CLASSES)
    model.to(config.DEVICE)
    
    # Create optimizer
    optimizer = optim.SGD(
        model.parameters(),
        lr=config.LEARNING_RATE,
        momentum=config.MOMENTUM,
        weight_decay=config.WEIGHT_DECAY
    )
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=config.LR_STEP_SIZE,
        gamma=config.LR_GAMMA
    )
    
    # Training loop
    best_loss = float('inf')
    train_losses = []
    valid_losses = []
    
    print(f"Starting training for {config.NUM_EPOCHS} epochs...")
    print(f"  - Batch size: {config.BATCH_SIZE} (with gradient accumulation: {config.GRADIENT_ACCUMULATION_STEPS})")
    print(f"  - Effective batch size: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS}")
    print(f"  - Image size: {config.IMG_SIZE}")
    print(f"  - Mixed precision: {config.MIXED_PRECISION}")
    print(f"  - Memory monitoring enabled")
    
    # Clear GPU cache before starting
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        initial_memory = torch.cuda.memory_allocated() / 1e9
        print(f"  - Initial GPU memory: {initial_memory:.2f}GB / 4.3GB")
    
    for epoch in range(config.NUM_EPOCHS):
        print(f"\n=== Epoch {epoch + 1}/{config.NUM_EPOCHS} ===")
        epoch_start_time = time.time()

        # Train for one epoch
        train_loss = train_one_epoch(
            model, optimizer, train_loader, config.DEVICE, epoch + 1, config.PRINT_FREQ
        )
        
        # Validate
        valid_loss = validate_model(model, valid_loader, config.DEVICE)
        
        # Update learning rate
        scheduler.step()
        
        # Calculate epoch time
        epoch_time = time.time() - epoch_start_time
        
        # Record losses
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        
        # Memory monitoring
        if torch.cuda.is_available():
            peak_memory = torch.cuda.max_memory_allocated() / 1e9
            torch.cuda.reset_peak_memory_stats()  # Reset for next epoch
        else:
            peak_memory = 0
        
        print(f"Epoch {epoch + 1} Summary:")
        print(f"  Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")
        print(f"  Time: {epoch_time:.1f}s, Peak GPU Memory: {peak_memory:.2f}GB")
        print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Memory warning
        if peak_memory > 4.0:
            print(f"    WARNING: High memory usage ({peak_memory:.2f}GB). Consider reducing batch size or image size.")
        
        # Save best model
        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'valid_loss': valid_loss,
                'config': config.__dict__  # Save all config settings
            }, 'best_faster_rcnn_model.pth')
            print(f"  ✅ New best model saved! Validation loss: {best_loss:.4f}")
        
        # Save checkpoint every few epochs
        if (epoch + 1) % config.SAVE_FREQ == 0:
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'valid_loss': valid_loss,
                'config': config.__dict__
            }, f'faster_rcnn_checkpoint_epoch_{epoch + 1}.pth')
    
    # Plot training curves    
    plt.close('all')
    
    # Create plot with error handling
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(train_losses, label='Training Loss', marker='o')
    ax.plot(valid_losses, label='Validation Loss', marker='s')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('Faster R-CNN Training Progress')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Save plot
    plt.tight_layout()
    plt.savefig('../results/faster_rcnn_evaluation/faster_rcnn_training_curves.png', dpi=100, bbox_inches='tight')
    print(f"✅ Training curves saved in '../results/faster_rcnn_evaluation/faster_rcnn_training_curves.png'")
    
    plt.close()
    
    return model

In [ ]:
# Main function
if __name__ == "__main__":
    # Verify data directory and yaml file exist
    if not config.DATA_ROOT.exists():
        print(f"Error: Data directory not found at {config.DATA_ROOT}")
        sys.exit(1)
    
    yaml_path = Path(config.DATA_YAML)
    if not yaml_path.exists():
        print(f"Error: data.yaml not found at {yaml_path}")
        sys.exit(1)
    
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    
    # Start training
    print("Starting Faster R-CNN training for AgroPest-12 dataset")
    print(f"Configuration:")
    print(f"  Classes: {config.NUM_CLASSES}")
    print(f"  Batch size: {config.BATCH_SIZE}")
    print(f"  Learning rate: {config.LEARNING_RATE}")
    print(f"  Epochs: {config.NUM_EPOCHS}")
    print(f"  Device: {config.DEVICE}")
    
    # Train the model
    trained_model = train_faster_rcnn()
    
    print("\nTraining completed! You can now:")
    print("1. Check 'best_faster_rcnn_model.pth' for the best model")
    print("2. View 'faster_rcnn_training_curves.png' for training progress")
    print("3. Run inference using the trained model")